In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('final_log_data.csv')

In [ ]:
df.head(5)

In [3]:
import glob
import time
import pandas as pd
# from xml.dom import minidom
from nltk import ngrams
from nltk.tokenize import sent_tokenize
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem import PorterStemmer
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\vmadmin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vmadmin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\vmadmin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
len(df['comment'])

In [ ]:
df['comment'][0]

In [4]:
normal = []
malicious = []
for i in range(len(df['comment'])):
    if df['Anomalous'][i] == 1:
        malicious.append(df['comment'][i])
    else:
        normal.append(df['comment'][i])



In [5]:
benign_sentence=[]
for i in normal:
    sentences=i.split('.')
    
    for sentence in sentences:
        benign_sentence.append(sentence)

In [6]:
stop_words = set(stopwords.words('english')) 

def fun_remove_stop_words(posts):

    filtered=''
    
    for x in posts.split(' '):
        if x not in stop_words:
            filtered+=' '+x
    
    return filtered

In [7]:
plain_text = normal

In [8]:
# convert from list to string

data=''
for x in plain_text:
    data+=" " + x

In [9]:

data=fun_remove_stop_words(data)  # remove stop words
data=data.split('.')              # split sentences

In [10]:

# seperate words inside tags

for i in range(len(data)):
    data[i]=data[i].replace('<', ' <')
    data[i]=data[i].replace('>', '> ')
    data[i]=data[i].replace('=', ' = ')

In [11]:
print("Benign records: %2i" %len(data))

Benign records: 10356


In [ ]:
cleaned_data

In [ ]:
type(data)

In [12]:
for val in data:
    if val == "":
        data.remove(val)

In [13]:
cleaner_data = []
for valu in data:
    cleaner_data.append(valu.replace("<br />  <br />", ""))

In [14]:
cleaned_data = []
for valu in cleaner_data:
    cleaned_data.append(valu.replace("\\", ""))

In [ ]:
len(cleaned_data)

In [15]:
final_normal = cleaned_data[:800]

In [ ]:
len(final_normal)

In [ ]:
malicious

In [18]:
final_values=[]
for i in final_normal:
    final_values.append((i,0))

In [19]:

# give labels to sql data


for i in malicious:
    final_values.append((i,1))

In [ ]:
len(final_values)

In [20]:
df=pd.DataFrame(final_values,columns=['comment','anomolous'])

In [ ]:
df.head(5)

In [21]:
len(df['comment'])

1000

In [22]:
df.to_csv('sqli.csv', index=False, encoding='utf-16')

In [23]:
df=pd.read_csv('sqli.csv',encoding='utf-16')

In [26]:
# vectorization of data

from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer( min_df=2, max_df=0.7, max_features=1444, stop_words=stopwords.words('english'))
posts = vectorizer.fit_transform(df['comment'].values.astype('U')).toarray()

In [64]:
with open('vectorizer_cnn_without_duration', 'wb') as fin:
    pickle.dump(vectorizer, fin)

In [27]:
posts.shape

(1000, 1444)

In [28]:
posts.shape=(1000,38,38,1)

In [29]:
X=posts

In [30]:
y=df['anomolous']

In [31]:
from sklearn.model_selection import train_test_split

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [33]:
X_test.shape

(200, 38, 38, 1)

In [35]:
trainX=X_train.copy()
trainX.shape=(X_train.shape[0],trainX.shape[1]*trainX.shape[2])
testX=X_test.copy()
testX.shape=(testX.shape[0],testX.shape[1]*testX.shape[2])

In [36]:
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB()

gnb.fit(trainX, y_train)

pred_gnb = gnb.predict(testX)

In [37]:
import pickle
with open('gnb_model_without_duration' , "wb") as f:
     pickle.dump(gnb, f)  

In [38]:
for i in range(len(pred_gnb)):
    if pred_gnb[i]>0.5:
        pred_gnb[i]=1
    elif pred_gnb[i]<=0.5:
        pred_gnb[i]=0

In [39]:
def accuracy_function(tp,tn,fp,fn):
    
    accuracy = (tp+tn) / (tp+tn+fp+fn)
    
    return accuracy

In [40]:
def precision_function(tp,fp):
    
    precision = tp / (tp+fp)
    
    return precision

In [41]:
def recall_function(tp,fn):
    
    recall=tp / (tp+fn)
    
    return recall

In [42]:
def confusion_matrix(truth,predicted):
    
    true_positive = 0
    true_negative = 0
    false_positive = 0
    false_negative = 0
    
    for true,pred in zip(truth,predicted):
        
        if true == 1:
            if pred == true:
                true_positive += 1
            elif pred != true:
                false_negative += 1

        elif true == 0:
            if pred == true:
                true_negative += 1
            elif pred != true:
                false_positive += 1
            
    accuracy=accuracy_function(true_positive, true_negative, false_positive, false_negative)
    precision=precision_function(true_positive, false_positive)
    recall=recall_function(true_positive, false_negative)
    
    return (accuracy,
            precision,
           recall)

In [43]:
accuracy,precision,recall=confusion_matrix(y_test,pred_gnb)

In [44]:
recall

0.9772727272727273

In [45]:
accuracy

0.98

In [47]:
precision

0.9347826086956522

In [48]:
import tensorflow as tf
from keras.models import Sequential
from keras import layers
from keras.preprocessing.text import Tokenizer
from keras.wrappers.scikit_learn import KerasClassifier

C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passi

In [52]:
model=tf.keras.models.Sequential([
    
    tf.keras.layers.Conv2D(64, (3,3), activation=tf.nn.relu, input_shape=(38,38,1)),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(128,(3,3), activation=tf.nn.relu),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(256,(3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dense(64, activation=tf.nn.relu),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [53]:
model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 36, 36, 64)        640       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 18, 18, 64)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 16, 16, 128)       73856     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 8, 8, 128)         0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 6, 6, 256)         295168    
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 3, 3, 256)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 2304)             

In [54]:
classifier_nn = model.fit(X_train,y_train,
                    epochs=10,
                    verbose=True,
                    validation_data=(X_test, y_test),
                    batch_size=128)

Train on 800 samples, validate on 200 samples
Epoch 1/10
800/800 [==============================] - 7s 9ms/sample - loss: 0.6043 - acc: 0.7200 - val_loss: 0.4953 - val_acc: 0.7800
Epoch 2/10
800/800 [==============================] - 6s 7ms/sample - loss: 0.4654 - acc: 0.8050 - val_loss: 0.4450 - val_acc: 0.7800
Epoch 3/10
800/800 [==============================] - 6s 8ms/sample - loss: 0.3829 - acc: 0.8438 - val_loss: 0.3419 - val_acc: 0.8700
Epoch 4/10
800/800 [==============================] - 6s 7ms/sample - loss: 0.2978 - acc: 0.8963 - val_loss: 0.2850 - val_acc: 0.8950
Epoch 5/10
800/800 [==============================] - 6s 8ms/sample - loss: 0.2132 - acc: 0.9187 - val_loss: 0.2159 - val_acc: 0.9150
Epoch 6/10
800/800 [==============================] - 6s 7ms/sample - loss: 0.1378 - acc: 0.9475 - val_loss: 0.1524 - val_acc: 0.9550
Epoch 7/10
800/800 [==============================] - 6s 7ms/sample - loss: 0.0770 - acc: 0.9825 - val_loss: 0.1047 - val_acc: 0.9750
Epoch 8/10
800/8

In [56]:
pred=model.predict(X_test)

In [57]:
for i in range(len(pred)):
    if pred[i]>0.5:
        pred[i]=1
    elif pred[i]<=0.5:
        pred[i]=0

In [58]:
accuracy,precision,recall=confusion_matrix(y_test,pred)

In [60]:
recall

0.9545454545454546

In [61]:
precision

0.8936170212765957

In [62]:
accuracy

0.965

In [63]:
model.save('cnn_model_without_duration.h5')

In [ ]:
def detect_malicious(val):
    
    def clean_data(val):
        val=val.replace('\n', '')
        val=val.replace('%20', ' ')
        val=val.replace('=', ' = ')
        val=val.replace('((', ' (( ')
        val=val.replace('))', ' )) ')
        val=val.replace('(', ' ( ')
        val=val.replace(')', ' ) ')
        val=val.replace('1 ', 'numeric')
        val=val.replace(' 1', 'numeric')
        val=val.replace("'1 ", "'numeric ")
        val=val.replace(" 1'", " numeric'")
        val=val.replace('1,', 'numeric,')
        val=val.replace(" 2 ", " numeric ")
        val=val.replace(' 3 ', ' numeric ')
        val=val.replace(' 3--', ' numeric--')
        val=val.replace(" 4 ", ' numeric ')
        val=val.replace(" 5 ", ' numeric ')
        val=val.replace(' 6 ', ' numeric ')
        val=val.replace(" 7 ", ' numeric ')
        val=val.replace(" 8 ", ' numeric ')
        val=val.replace('1234', ' numeric ')
        val=val.replace("22", ' numeric ')
        val=val.replace(" 8 ", ' numeric ')
        val=val.replace(" 200 ", ' numeric ')
        val=val.replace("23 ", ' numeric ')
        val=val.replace('"1', '"numeric')
        val=val.replace('1"', '"numeric')
        val=val.replace("7659", 'numeric')
        val=val.replace(" 37 ", ' numeric ')
        val=val.replace(" 45 ", ' numeric ')
        
        return val
    
    val=clean_data(val)
    val=[val]
    val=vectorizer.transform(val).toarray()
    result=gnb.predict(val)
    
    if result>0.5:
        print("ALERT :::: This can be SQL injection")
    elif result<=0.5:
        print("It seems to be safe")

In [ ]:
df_full = pd.read_csv("./data_comments/original_datasets/sqli.csv", encoding='utf-16')

In [ ]:
test_comments = df_full['Sentence'][101:150]

In [ ]:
for comm in test_comments:
    detect_malicious(comm)

In [ ]:
c1=''' )  union select * from information_schema.tables'''

In [ ]:
input_val=clean_data(c1)
input_val=[input_val]

In [ ]:
input_val=vectorizer.transform(input_val).toarray()

In [ ]:
result=gnb.predict(input_val)

In [ ]:
if result>0.5:
    print("ALERT :::: This can be SQL injection")
elif result<=0.5:
    print("It seems to be safe")

In [ ]:
comm = ['This is nice movie']

In [ ]:
input_val=clean_data(comms_vectorized)
input_val=[input_val]

In [ ]:
comms_vectorized = vectorizer.transform(comm).toarray()

In [ ]:
comms_vectorized.shape

In [ ]:
result=gnb.predict(comms_vectorized)

In [ ]:
result

In [ ]:
if result>0.5:
    print("ALERT :::: This can be SQL injection")
elif result<=0.5:
    print("It seems to be safe")

In [ ]:
c2 = r" AND 1 = utl_inaddr.get_host_address  (  (  SELECT DISTINCT ( USERNAME )  FROM  ( SELECT DISTINCT ( USERNAME ) , ROWNUM AS LIMIT FROM SYS.ALL_USERS )  WHERE LIMIT = 1  )  )   AND 'i' = 'i'"

In [ ]:
c2

In [ ]:
input_val=clean_data(c2)
input_val=[input_val]

In [ ]:
input_val=vectorizer.transform(input_val).toarray()

In [ ]:
result=gnb.predict(input_val)

In [ ]:
if result>0.5:
    print("ALERT :::: This can be SQL injection")
elif result<=0.5:
    print("It seems to be safe")

In [ ]:
from keras.models import load_model
import pickle

gnb.save('gnb_model.h5')

In [ ]:
import keras
from keras.models import load_model
import pickle

mymodel = load_model('my_model.h5')
myvectorizer = pickle.load(open("vectorizer", 'rb'))


def predict_sqli_attack():
	
	beautify=''
	for i in range(20):
        beautify+= "="

	print(beautify) 
	input_val=input("Give me some data to work on : ")
	print(beautify)

	def clean_data(input_val):    

	    input_val=input_val.replace('\n', '')
	    input_val=input_val.replace('%20', ' ')
	    input_val=input_val.replace('=', ' = ')
	    input_val=input_val.replace('((', ' (( ')
	    input_val=input_val.replace('))', ' )) ')
	    input_val=input_val.replace('(', ' ( ')
	    input_val=input_val.replace(')', ' ) ')
	    input_val=input_val.replace('1 ', 'numeric')
	    input_val=input_val.replace(' 1', 'numeric')
	    input_val=input_val.replace("'1 ", "'numeric ")
	    input_val=input_val.replace(" 1'", " numeric'")
	    input_val=input_val.replace('1,', 'numeric,')
	    input_val=input_val.replace(" 2 ", " numeric ")
	    input_val=input_val.replace(' 3 ', ' numeric ')
	    input_val=input_val.replace(' 3--', ' numeric--')
	    input_val=input_val.replace(" 4 ", ' numeric ')
	    input_val=input_val.replace(" 5 ", ' numeric ')
	    input_val=input_val.replace(' 6 ', ' numeric ')
	    input_val=input_val.replace(" 7 ", ' numeric ')
	    input_val=input_val.replace(" 8 ", ' numeric ')
	    input_val=input_val.replace('1234', ' numeric ')
	    input_val=input_val.replace("22", ' numeric ')
	    input_val=input_val.replace(" 8 ", ' numeric ')
	    input_val=input_val.replace(" 200 ", ' numeric ')
	    input_val=input_val.replace("23 ", ' numeric ')
	    input_val=input_val.replace('"1', '"numeric')
	    input_val=input_val.replace('1"', '"numeric')
	    input_val=input_val.replace("7659", 'numeric')
	    input_val=input_val.replace(" 37 ", ' numeric ')
	    input_val=input_val.replace(" 45 ", ' numeric ')
	    
	    return input_val

	input_val=clean_data(input_val)
	input_val=[input_val]



	input_val=myvectorizer.transform(input_val).toarray()

	result=mymodel.predict(input_val)


	print(beautify)
	if result>0.5:
		print("ALERT :::: This can be SQL injection")
	elif result<=0.5:
		print("It seems to be safe")

	print(beautify)


predict_sqli_attack()